In [43]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [44]:
# Write A function to create complementary color image
def complementary_color_RGB(img):
    # Split the image into three channels
    b, g, r = cv.split(img)

    b = cv.bitwise_not(b)
    g = cv.bitwise_not(g)
    r = cv.bitwise_not(r)

    # Merge the three channels
    img = cv.merge([b, g, r])

    return img

In [45]:
def complementary_color_HSI(img):
    h, s, i = img[:, :, 0], img[:, :, 1], img[:, :, 2]

    h = (h + 0.5) % 1.0

    return np.dstack((h, s, i))

In [46]:
def convertRGB2HSI(img):

    # Convert RGB to HSI
    width, height, channel = img.shape

    # Create a new image
    new_img = np.zeros((width, height, channel), dtype=np.float64)

    for i in range(width):
        for j in range(height):
            # Get RGB value
            red, green, blue = img[i, j, 2], img[i, j, 1], img[i, j, 0]
            red, green, blue = red / 255.0, green / 255.0, blue / 255.0

            # Convert RGB to HSI
            intensity = max([red, green, blue])
            delta = intensity - min([red, green, blue])

            if intensity == 0:
                saturation = 0
                hue = 0
            elif delta == 0:
                saturation = 0
                hue = 0
            else:
                saturation = delta / intensity

                hue = 0
                if red == intensity:
                    hue = (1.0/6.0) * (green - blue) / delta
                elif green == intensity:
                    hue = (1.0/6.0) * (2.0 + ((blue - red) / delta))
                elif blue == intensity:
                    hue = (1.0/6.0) * (4.0 + ((red - green) / delta))

            new_img[i, j, 0] = hue
            new_img[i, j, 1] = saturation
            new_img[i, j, 2] = intensity

    return new_img

In [47]:
def convertHSI2RGB(img):

    # Convert HSI to RGB
    width, height, channel = img.shape

    # Create a new image
    new_img = np.zeros((width, height, channel), dtype=np.uint8)

    for i in range(width):
        for j in range(height):
            # Get HSI value
            hue, saturation, intensity = img[i, j]

            r, g, b = 0, 0, 0

            H_prime = math.floor(hue * 6)
            F = (hue * 6) - H_prime
            P = intensity * (1 - saturation)
            Q = intensity * (1 - (F * saturation))
            T = intensity * (1 - ((1 - F) * saturation))

            if H_prime == 0:
                r = intensity
                g = T
                b = P
            elif H_prime == 1:
                r = Q
                g = intensity
                b = P
            elif H_prime == 2:
                r = P
                g = intensity
                b = T
            elif H_prime == 3:
                r = P
                g = Q
                b = intensity
            elif H_prime == 4:
                r = T
                g = P
                b = intensity
            elif H_prime == 5:
                r = intensity
                g = P
                b = Q

            new_img[i, j] = [r * 255, g * 255, b * 255]

    return new_img

In [48]:
# Remove the output folder if it exists
if os.path.exists("output/"):
    os.system("rm -rf output/")

os.mkdir("output/")
os.mkdir("output/5_1")

# Read image
fruit_path = "img/fruit.jpg"
fruit = cv.imread(fruit_path, cv.IMREAD_COLOR)
fruit_rgb = cv.cvtColor(fruit, cv.COLOR_BGR2RGB)

print("Original image")
print(fruit_rgb[0, 0, :])

fruit_hsi = convertRGB2HSI(fruit)

fruit_rgb = convertHSI2RGB(fruit_hsi)

fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(cv.cvtColor(fruit, cv.COLOR_BGR2RGB))
ax[0].set_title("Original image")

ax[1].imshow(fruit_rgb)
ax[1].set_title("Converted image")

plt.show()




Original image
[183  65   3]


ValueError: could not broadcast input array from shape (765,) into shape (3,)